# Infuse Competitive Programming with Code Completion
# Rayyan Khan, Abid Rahman

# Setup

In [ ]:
!pip install transformers accelerate datasets memory_profiler evaluate fire --quiet
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.2 MB/s eta 0:00:00
   

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from huggingface_hub import login
import os
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

# Load Models

In [ ]:
# (Optional but helpful) reduce fragmentation — set BEFORE importing torch/transformers
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# If needed:
# !pip install -q bitsandbytes accelerate transformers

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/deepseek-coder-6.7b-base"  # single source of truth

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # or torch.bfloat16 if your GPU supports bf16
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,  # preferred over passing individual 4bit kwargs
    low_cpu_mem_usage=True,
    # If you still OOM, uncomment the next line to hard-cap GPU memory and spill to CPU:
    # max_memory={0: "14500MB", "cpu": "30GB"},
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
import textwrap
from transformers import StoppingCriteria, StoppingCriteriaList

# --------------------------
# Custom stopping criteria
# --------------------------
class StopOnSeq(StoppingCriteria):
    def __init__(self, tokenizer, seqs):
        self.stop_ids = [tokenizer(s, add_special_tokens=False).input_ids for s in seqs]
    def __call__(self, input_ids, scores, **kwargs):
        for s in self.stop_ids:
            if len(s) <= input_ids.shape[-1] and input_ids[0, -len(s):].tolist() == s:
                return True
        return False

stopping = StoppingCriteriaList([
    StopOnSeq(tokenizer, ["```", "\n\n"])  # stop on code fence or blank line
])

# --------------------------
# Prompt
# --------------------------
prompt = (
    "Write ONLY the Python function is_prime(n) that returns True if n is prime, else False.\n"
    "Do not add any other text or functions.\n"
    "```python\n"
    "def is_prime(n):\n"
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --------------------------
# Generation
# --------------------------
with torch.no_grad():
    generated = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=stopping
    )

# --------------------------
# Post-processing
# --------------------------
new_tokens = generated[0, inputs["input_ids"].shape[1]:]
body = tokenizer.decode(new_tokens, skip_special_tokens=True)

# Remove stray triple backticks if still present
body = body.split("```")[0]

# Dedent and strip leading/trailing blank lines
body = textwrap.dedent(body).strip()

# Print full function only
print(f"def is_prime(n):\n{textwrap.indent(body, '    ')}")


def is_prime(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True


# Load in Dataset

In [3]:
from datasets import load_dataset
import pandas as pd


In [5]:
ds = load_dataset("newfacade/LeetCodeDataset", split="train")

df = pd.DataFrame(ds)
print(df.head())

                                          task_id  question_id difficulty  \
0                                         two-sum            1       Easy   
1                                 add-two-numbers            2     Medium   
2  longest-substring-without-repeating-characters            3     Medium   
3                     median-of-two-sorted-arrays            4       Hard   
4                   longest-palindromic-substring            5     Medium   

                                          tags  \
0                          [Array, Hash Table]   
1               [Recursion, Linked List, Math]   
2         [Hash Table, String, Sliding Window]   
3   [Array, Binary Search, Divide and Conquer]   
4  [Two Pointers, String, Dynamic Programming]   

                                 problem_description  \
0  Given an array of integers nums and an integer...   
1  You are given two non-empty linked lists repre...   
2  Given a string s, find the length of the longe...   
3  Given two

#Test Model

In [ ]:
def run_model():
